In [1]:
# Initialisation

#q1 of ass1

import numpy as np
import math
import torch as t
import torch 
import os

log_dir = '../../log/'
os.makedirs(log_dir, exist_ok=True)  
logfile = os.path.join(log_dir, 'q1.log')

import logging

logging.basicConfig(filename=logfile, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

%matplotlib inline
# import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
import matplotlib.pyplot as plt


In [2]:

# part 1 and 2:
# dataset loading:
training_data = datasets.FashionMNIST(
    root="../../data/",
    train=True,
    download=True,
    transform=ToTensor() # Normalisation step
    
)

# note that ToTensor not just converts the image into a tensor but also normalises its intensity in range 0 to 1


test_data = datasets.FashionMNIST(
    root="../../data/",
    train=False,
    download=True,
    transform=ToTensor() # Normalisation step
)

# Create data loaders for training and testing

batchSize=64
train_loader = DataLoader(training_data, batch_size=batchSize, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batchSize, shuffle=False)

In [3]:
# part 3:

import torch.nn as nn

# builtin classifier:

# Define the model

# model saving
# Define a directory to save models
save_dir = '../../saved_models/q1/'
os.makedirs(save_dir, exist_ok=True)

model = nn.Sequential(

    # we need to flatten the input data from 2d tensor into 1d tensor of 784 values (28*28) since data is in form of images
    nn.Flatten(),  # Flatten the input data

    nn.Linear(in_features=784, out_features=10),  # Linear layer   
    # nn.Linear(in_features=784, out_features=batchSize),  # Linear layer   

    # softmax function, or normalized exponential function converts a vector of K real numbers into a probability distribution of K possible outcomes
    nn.Softmax(dim=1)  # Softmax activation
)

# Specify the path to the saved checkpoint file
# model_path = '../../saved_models/model_checkpoint.pth'
model_path = os.path.join(save_dir, 'model_checkpoint.pth')

# Check if the file exists before loading
if os.path.exists(model_path):
    # Load the model checkpoint
    checkpoint = torch.load(model_path)

    # Load the model's state_dict
    model.load_state_dict(checkpoint)
    print("Model loaded successfully.")
else:
    print(f"Checkpoint file '{model_path}' not found. Model not loaded.")

# custom made classifier
'''
class LinearClassifier(nn.Module):
    def __init__(self):
        super(LinearClassifier, self).__init__()

        # Define the linear layer
        # we used 28*28 input and 10 labels because we are working with MNIST database
        self.fc = nn.Linear(28 * 28, 10)  # 28*28 input features, 10 output classes
        
        # Define an activation function (e.g., softmax for classification)

        # softmax function, or normalized exponential function converts a vector of K real numbers into a probability distribution of K possible outcomes
        self.softmax = nn.Softmax(dim=1)

    #IMPORTANT the name 'forward' is not arbitrary; it's a convention in PyTorch, and this method is automatically called when we pass data through our model.
    def forward(self, x):

        # we need to flatten the input data from 2d tensor into 1d tensor of 784 values (28*28) since data is in form of images
        x = x.view(x.size(0), -1)  # Flatten the input tensor

        x = self.fc(x)  # Apply the linear transformation
        x = self.softmax(x) # Apply the activation function
        return x
'''
print('model created')

Model loaded successfully.
model created


In [4]:
print("Model structure before training: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure before training:  Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=10, bias=True)
  (2): Softmax(dim=1)
) 


Layer: 1.weight | Size: torch.Size([10, 784]) | Values : tensor([[ 0.0096, -0.0067, -0.0217,  ...,  0.0004, -0.0380, -0.0330],
        [-0.0126,  0.0191, -0.0123,  ..., -0.0116,  0.0045,  0.0258]],
       grad_fn=<SliceBackward0>) 

Layer: 1.bias | Size: torch.Size([10]) | Values : tensor([ 0.0508, -0.0041], grad_fn=<SliceBackward0>) 



In [5]:
# part 4:

# Define the loss function
criterion = nn.CrossEntropyLoss()

import torch.optim as optim
# Define the optimizer (e.g., SGD)

learning_rate=0.01
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

'''
about SGD: Stochastic Gradient Descent:
basically it performs seqeuntial updation of paramters after each epoch, using formula :
parameter = parameter - learning_rate * gradient

'''

print('part 4 done')



part 4 done


In [6]:
# part 5
# Define the number of training epochs
num_epochs = 1

import time
t1=time.time()
# Training loop
for epoch in range(num_epochs):
    # Set the model in training mode
    model.train()
    
    running_loss = 0.0
    
    # Iterate over the training dataset
    for inputs, labels in train_loader:  # Replace train_loader with your data loader
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        
        # Compute the loss
        loss = criterion(outputs, labels)
        
        # Backpropagation
        loss.backward()
        
        # Update the model's parameters
        optimizer.step()
        
        # Track the running loss
        running_loss += loss.item()
    
    # Calculate and print the average loss for this epoch
    average_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {average_loss:.4f}")

print("Training complete")
t2=time.time()
training_time=t2-t1
print('time taken to train : ',training_time)



# Save the model checkpoint to the specified directory
# model_path = os.path.join(save_dir, 'model_checkpoint.pth')
torch.save(model.state_dict(), model_path)

Epoch [1/1] - Loss: 1.7943
Training complete
time taken to train :  8.71134090423584


In [7]:
print("Model structure after training: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure after training:  Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=10, bias=True)
  (2): Softmax(dim=1)
) 


Layer: 1.weight | Size: torch.Size([10, 784]) | Values : tensor([[ 0.0096, -0.0067, -0.0217,  ..., -0.0004, -0.0383, -0.0330],
        [-0.0126,  0.0191, -0.0123,  ..., -0.0117,  0.0045,  0.0258]],
       grad_fn=<SliceBackward0>) 

Layer: 1.bias | Size: torch.Size([10]) | Values : tensor([ 0.0537, -0.0082], grad_fn=<SliceBackward0>) 



In [8]:
# part 6 :
# testing 

# Set the model to evaluation mode
model.eval()

correct_predictions = 0
total_predictions = 0

# Iterate over the testing dataset

k=0
for inputs, labels in test_loader:  # Use your test data loader
    k+=1
    # Forward pass to obtain predictions
    outputs = model(inputs)
    
    # Get the predicted class for each example in the batch
    _, predicted = torch.max(outputs, dim=1)
    
    # Count correct predictions in this batch
    batch_correct_predictions = (predicted == labels).sum().item()
    correct_predictions += batch_correct_predictions
    
    # Count total predictions in this batch

    batch_total_predictions = labels.size(0)
    total_predictions += batch_total_predictions 

    # Print the running accuracy for this batch
    # Update running accuracy
    batch_accuracy=batch_correct_predictions/batch_total_predictions * 100.0
    print(f"Accuracy on this batch {batch_accuracy:.2f}%")

    accuracy = (correct_predictions / total_predictions) * 100.0
    print(f"Accuracy till now:  testing dataset: {accuracy:.2f}%")

print()
print("number of tests:",k)

# Calculate the accuracy
accuracy = (correct_predictions / total_predictions) * 100.0
print(f"Accuracy on the testing dataset: {accuracy:.2f}%")
print('time taken to train : ',training_time)

# saving accuracy
import logging


# Your accuracy calculation
# accuracy = 0.85  # Replace this with your actual accuracy value

# Log the accuracy
log_dir = '../../log'
os.makedirs(save_dir, exist_ok=True)
logfile=os.path.join(log_dir, 'q1.log')
logging.basicConfig(filename=logfile, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Configure logging
# logging.basicConfig(filename='../../log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


logging.info(f"Accuracy: {accuracy:.2f}%")
logging.info(f"Training Time: {training_time:.2f} seconds")
logging.info(f"Number of epochs: {num_epochs:.2f} seconds")
print("data saved")



Accuracy on this batch 62.50%
Accuracy till now:  testing dataset: 62.50%
Accuracy on this batch 79.69%
Accuracy till now:  testing dataset: 71.09%
Accuracy on this batch 76.56%
Accuracy till now:  testing dataset: 72.92%
Accuracy on this batch 76.56%
Accuracy till now:  testing dataset: 73.83%
Accuracy on this batch 71.88%
Accuracy till now:  testing dataset: 73.44%
Accuracy on this batch 78.12%
Accuracy till now:  testing dataset: 74.22%
Accuracy on this batch 59.38%
Accuracy till now:  testing dataset: 72.10%
Accuracy on this batch 73.44%
Accuracy till now:  testing dataset: 72.27%
Accuracy on this batch 68.75%
Accuracy till now:  testing dataset: 71.88%
Accuracy on this batch 65.62%
Accuracy till now:  testing dataset: 71.25%
Accuracy on this batch 62.50%
Accuracy till now:  testing dataset: 70.45%
Accuracy on this batch 65.62%
Accuracy till now:  testing dataset: 70.05%
Accuracy on this batch 70.31%
Accuracy till now:  testing dataset: 70.07%
Accuracy on this batch 76.56%
Accuracy

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\pande\\OneDrive\\Desktop\\code\\ML\\ELL409-Machine-Learning-Coursework\\assignment-01\\log\\q1.log'